## Задание 1

$$\int_{-1}^1\frac{\sin(\frac{x}{2})}{e^x-1}dx$$

In [1]:
from decimal import Decimal, getcontext
import math
from functools import lru_cache

###############################################################################
#  1) Вспомогательные функции для разностных квадратур                        #
###############################################################################
getcontext().prec = 100  # повышенная точность для Decimal

@lru_cache(None)
def factorial_dec(n: int) -> Decimal:
    if n < 0:
        raise ValueError("Negative factorial not defined")
    if n == 0 or n == 1:
        return Decimal(1)
    return Decimal(n) * factorial_dec(n - 1)

@lru_cache(None)
def comb_dec(n: int, k: int) -> Decimal:
    if k < 0 or k > n:
        return Decimal(0)
    return factorial_dec(n) / (factorial_dec(k) * factorial_dec(n - k))

@lru_cache(None)
def D(n: int, j: int) -> Decimal:
    """
    Вычисляет D_n^(j) по заданной рекуррентной формуле (см. теорию).
    """
    if j == 0:
        return Decimal(1)
    # "прогреваем" кэш:
    _ = D(n, j-1)
    return _compute_all_D_up_to(n, j)[j]

@lru_cache(None)
def _compute_all_D_up_to(n: int, J: int):
    results = [Decimal(0)]*(J+1)
    results[0] = Decimal(1)
    for j in range(J):
        s = Decimal(0)
        for k in range(n + 2*j + 1):
            for l in range(j+1):
                val = (
                    Decimal((-1)**k)
                    * results[l]
                    * comb_dec(n+2*l, k - (j - l))
                    * Decimal((n + 2*j - 2*k)**(n + 2*j + 2))
                    / (Decimal(2)**(n + 2*j + 2) * factorial_dec(n + 2*j + 2))
                )
                s += val
        # Финальная формула берёт только s, без + results[j].
        results[j+1] = s
    return results

@lru_cache(None)
def A(k: int, n: int, m: int) -> Decimal:
    """
    A_{k,n}^m = sum_{l=0}^m [ (-1)^(k-m) * D_n^(l) * C_{n+2l}^{k - m + l} ].
    """
    s = Decimal(0)
    sign = Decimal((-1)**(k - m))
    for l in range(m+1):
        s += (
            sign
            * D(n, l)
            * comb_dec(n+2*l, k - m + l)
        )
    return s

@lru_cache(None)
def W_list(m: int):
    """
    Коэффициенты W_k^m (k=0..2m) в Decimal.

      W_k^m = sum_{n=0}^m [ A_{k,2n}^{m-n} / (2^(2n)*(2n+1)!) ].
    """
    result = []
    for k in range(2*m + 1):
        s = Decimal(0)
        for n_ in range(m+1):
            a_val = A(k, 2*n_, m - n_)
            denom = (Decimal(2)**(2*n_) * factorial_dec(2*n_+1))
            s += a_val / denom
        result.append(s)
    return result

def build_function_values_diff_scheme(f, a: float, b: float, J: int, m: int):
    """
    Центральная сетка: x_j = a + (j+0.5)*h, j=-m..(J-1+m).
    Возвращает (y_values, h, calls).
    """
    h = (b - a)/J
    values = []
    for j_real in range(-m, (J - 1) + m + 1):
        x_val = a + (j_real + 0.5)*h
        values.append(f(x_val))
    calls = len(values)
    return values, h, calls

def integrate_by_diff_scheme(f, a: float, b: float, J: int, m: int):
    """
    Разностная (дифференсная) квадратура порядка 2m:
      ∫[a..b] f(x) dx ≈ h Σ_j Σ_k [ W_{m-k} * f( x_{j+k} ) ].
    Возвращает (approx, calls).
    """
    y_values, h, calls = build_function_values_diff_scheme(f, a, b, J, m)
    W_dec = W_list(m)
    W = [float(wd) for wd in W_dec]
    
    offset = m
    total_sum = 0.0
    for j in range(J):
        loc_sum = 0.0
        for k in range(-m, m+1):
            idx = (j + k) + offset
            loc_sum += W[m - k] * y_values[idx]
        total_sum += loc_sum
    return (h * total_sum, calls)


In [2]:
def f1(x: float) -> float:
        return math.sin(x/2)/(math.exp(x)-1)

In [4]:
integrate_by_diff_scheme(f1, -1, 1, 10, 7)[0]

1.0130392362326262